In [1]:
import matplotlib as mpl #画图用的库
import matplotlib.pyplot as plt
#下面这一句是为了可以在notebook中画图
%matplotlib inline
import numpy as np
import sklearn   #机器学习算法库
import pandas as pd #处理数据的库   
import os
import sys
import time
import tensorflow as tf
 
from tensorflow import keras   #使用tensorflow中的keras
#import keras #单纯的使用keras
 
print(tf.__version__)
print(sys.version_info)
for module in mpl, np, sklearn, pd, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.18.0
sklearn 0.21.3
pandas 0.25.3
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
#from_tensor_slices 说白了就是 用来初始化数据集的
#从内存中构建生成数据集,感觉类似于C/C++中的数组指针的感觉
#传入 numpy数组,来初始化dataset
dataset = tf.data.Dataset.from_tensor_slices(np.arange(10))#传入numpy数组，数组大小为长度为10的一维向量
print(dataset)
for item in dataset:
    print(item)

<TensorSliceDataset shapes: (), types: tf.int64>
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


In [3]:
#使用dataset实现以下的两种功能
#1.repeat epoch :表示数据集读取遍历次数，遍历一次即epoch=1，遍历n次数据集即epoch=n
#2.get batch :batch_size表示训练网络数据参数时取数据集中的 batch_size个 参数进入到模型中进行训练
dataset = dataset.repeat(3).batch(7) #repeat(3)表示这个一维向量会被遍历三次 batch(7)表示batch_size为7，即每次取7个数据
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int64)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int64)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int64)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int64)
tf.Tensor([8 9], shape=(2,), dtype=int64)


In [4]:
#interleave: 中文意思是 交叉，主要应用于生成新的数据集，最终的目的是为了数据的均匀混合作用
#其改变了数据集中数据排列方式，但是不会对数据做增减等修改，所以个人认为不需要太过于关注 cycle_length和block_length的取值
#案例: 文件dataset --> 具体数据集
dataset2 = dataset.interleave(
    lambda v: tf.data.Dataset.from_tensor_slices(v), #map_fn
    cycle_length = 4, #cycle_length 表示 每次从dataset中取出 cycle_length个元素，然后对这些元素应用map_fn，得到cycle_length个新的Dataset对象
    block_length = 4, #block_length 表示 从上面生成的新的datasets对象中轮流从其中取元素，每次取block_length个
)
for item in dataset2:
    print(item.numpy(),end=', ')
    ##这里在取了四行之后(因为cycle_length=4)，就重头回去取第一行(先不取最后一行8, 9)

0, 1, 2, 3, 7, 8, 9, 0, 4, 5, 6, 7, 1, 2, 3, 4, 4, 5, 6, 1, 2, 3, 8, 9, 0, 5, 6, 7, 8, 9, 

In [5]:
#传入 元组，来初始化dataset3
x = np.array([[1, 2], [3, 4], [5, 6]])#x 和 y的维度必须相等
y = np.array(['cat', 'dog', 'fox'])
dataset3 = tf.data.Dataset.from_tensor_slices((x, y))
print(dataset3)

for item_x, item_y in dataset3:
    print(item_x, item_y)
    print(item_x.numpy(), item_y.numpy())

<TensorSliceDataset shapes: ((2,), ()), types: (tf.int64, tf.string)>
tf.Tensor([1 2], shape=(2,), dtype=int64) tf.Tensor(b'cat', shape=(), dtype=string)
[1 2] b'cat'
tf.Tensor([3 4], shape=(2,), dtype=int64) tf.Tensor(b'dog', shape=(), dtype=string)
[3 4] b'dog'
tf.Tensor([5 6], shape=(2,), dtype=int64) tf.Tensor(b'fox', shape=(), dtype=string)
[5 6] b'fox'


In [6]:
#传入 字典,来初始化 dataset4
dataset4 = tf.data.Dataset.from_tensor_slices({"feature" : x,
                                               "label"   : y})
for item in dataset4:
    #print(item)
    print(item["feature"].numpy(), item["label"].numpy())

[1 2] b'cat'
[3 4] b'dog'
[5 6] b'fox'
